In [3]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import time
# from haversine import haversine_vector, Unit
# from haversine import haversine

In [4]:
def haverDist(OHCA_Cluster,AED_Cluster):
    d = haversine_vector(list(zip(OHCA_Cluster.LatOHCA, OHCA_Cluster.LongOHCA)), list(zip(AED_Cluster.LatAED, AED_Cluster.LongAED)), Unit.METERS, comb=True).T
    return(d)
#zip()creates the objects and that can be used to produce single item at a time.
#This function can create tuples by merging two lists

In [17]:
SGbuildings = pd.read_csv('../../../OneDrive/Documents/University_Files/Y4S1/IE3100M - System Design Project/Li Wanbing Documents/Virtual Test Problems/Singapore buildings locations.csv')
# SGbuildings = pd.read_csv('../../../OneDrive/Documents/University_Files/Y4S1/IE3100M - System Design Project/Li Wanbing Documents/Virtual Test Problems/Singapore buildings locations.csv').rename(columns={'Lat':'LatAED','Lon':'LongAED'})
# trainingOHCA = pd.read_csv('../../../OneDrive/Documents/University_Files/Y4S1/IE3100M - System Design Project/Li Wanbing Documents/Virtual Test Problems/Test Problem Set 1 - 20210203/generatedPoints1_100000.csv',names=['LatOHCA','LongOHCA'])

# SGbuildings.head()

FileNotFoundError: [Errno 2] File b'../../../OneDrive/Documents/University_Files/Y4S1/IE3100M - System Design Project/Li Wanbing Documents/Virtual Test Problems/Singapore buildings locations.csv' does not exist: b'../../../OneDrive/Documents/University_Files/Y4S1/IE3100M - System Design Project/Li Wanbing Documents/Virtual Test Problems/Singapore buildings locations.csv'

In [ ]:
haverDist(trainingOHCA.head(3), candidateAED.head())
#returns a np.array of shape(no. of OHCA points, no. of AED points)，hence set (d[i][j]<S) as a constaint in MCLP
#to verify: https://www.vcalc.com/wiki/vCalc/Haversine+-+Distance

In [ ]:
trainingOHCA.head()

In [ ]:
SGbuildings.shape

In [ ]:
def kmeans_OHCA_AED(OHCA,AED,k): #k is the number of clusters chosen
    kmeans = KMeans(n_clusters=k, random_state=0).fit(OHCA)
    
    #labels_:ndarray of shape (n_samples,), gives the label of cluster that each OHCA belongs to
    OHCA_Cluster_All = pd.concat([OHCA,pd.DataFrame(kmeans.labels_,columns = ["cluster"])],axis=1)
    
    AED_label = kmeans.predict(AED)
    AED_Cluster_All = pd.concat([AED, pd.DataFrame(AED_label,columns = ["cluster"])], axis=1)
    return OHCA_Cluster_All, AED_Cluster_All, kmeans.cluster_centers_

In [ ]:
%%time
start_time = time.time()
print('Splitting into 55 clusters:')
OHCA_55cluster, buildings_55cluster, CC_55 = kmeans_OHCA_AED(trainingOHCA, SGbuildings, 55)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
np.sort(OHCA_55cluster['cluster'].unique())

In [ ]:
print(OHCA_55cluster.shape)
print('Number of NaNs:',OHCA_55cluster['cluster'].isna().sum())


print(buildings_55cluster.shape)
print('Number of NaNs:', buildings_55cluster['cluster'].isna().sum())


In [ ]:
OHCA_55cluster.groupby(['cluster'])[['LatOHCA']].count().rename(columns={'LatOHCA':'Number of OHCA in cluster'})

In [ ]:
%%time
start_time = time.time()

cluster_centers = []
OHCA_9900cluster = []
buildings_9900cluster = []

print('Number of NaNs in each cluster:')
for i in range(0,55):    
    OHCA_i = OHCA_55cluster.loc[OHCA_55cluster['cluster']==i].drop(columns=['cluster'])
    buildings_i = buildings_55cluster.loc[buildings_55cluster['cluster']==i].drop(columns=['cluster'])
    
    #assume 9880 current AEDs, 9880/55 = approx 180
    OHCA_180cluster, buildings_180cluster, CC_180 = kmeans_OHCA_AED(OHCA_i, buildings_i, 180) 
    NanCount = OHCA_180cluster['cluster'].isna().sum()
    
    cluster_centers.append(CC_180)
    OHCA_9900cluster.append(OHCA_180cluster)
    buildings_9900cluster.append(AED_180cluster)
    
    print(OHCA_180cluster.shape, buildings_180cluster.shape)
    print('Cluster {}:'.format(i), NanCount)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
lst_for_df = []
for i in range(0,55):
    cluster_i = pd.DataFrame(cluster_centers[i],columns=['LatAED','LongAED'])
    lst_for_df.append(cluster_i)


In [ ]:
final_CC_df = pd.concat(lst_for_df, keys=list(i for i in range(0,55)))

In [ ]:
final_CC_df

In [ ]:
final_CC_df.isna().sum()

In [ ]:
#Example: cluster 0
OHCA_0 = OHCA_55cluster.loc[OHCA_55cluster['cluster']==0].drop(columns=['cluster'])
buildings_0 = buildings_55cluster.loc[buildings_55cluster['cluster']==0].drop(columns=['cluster'])
OHCA_180, buildings_180, CC = kmeans_OHCA_AED(OHCA_0, buildings_0, 180) 
OHCA_180.info()

In [ ]:
def totalCoverage(ohca_df, aed_df, MAX_DIST_METERS = 100):

    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5
        
        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000
    
    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 100, 'isCovered'] = 'covered'
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] > 100,'isCovered'] = 'uncovered'
    # each OHCA having an AED within 100m has isCovered = 'covered', otherwise 'uncovered'
    #print(cp_ohca)
    
    
    within_100m  = sum(cp_ohca['isCovered'] == 'covered')
    outside_100m = len(cp_ohca) - within_100m
    total_coverage_within_100m = within_100m / (within_100m + outside_100m)
    print("total coverage = ", total_coverage_within_100m)
    
    return total_coverage_within_100m

def partial_coverage(ohca_df, aed_df, ALPHA = 0.05):
    def ClosestAED(r):
        # Cartesian Distance: square root of (x2-x1)^2 + (y2-y1)^2
        distances = ((r['LatOHCA']-aed_df['LatAED'])**2 + (r['LongOHCA']-aed_df['LongAED'])**2)**0.5

        # AED with minimum Distance from the OHCA
        closestAEDId = distances[distances == distances.min()].index.to_list()[0]
        return aed_df.loc[closestAEDId, ['LatAED', 'LongAED']]

    cp_ohca = ohca_df.copy()
    cp_ohca[['ClosestAEDLatitude', 'ClosestAEDLongtitude']] = cp_ohca.apply(ClosestAED, axis=1)
    
    def haversine_np(lat1, lon1, lat2, lon2):
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km * 1000

    cp_ohca['dist_nearest_AED'] = haversine_np(cp_ohca['LatOHCA'], cp_ohca['LongOHCA'],
                                               cp_ohca['ClosestAEDLatitude'], cp_ohca['ClosestAEDLongtitude'])
    # distance to nearest AED (in metres) is recorded for each OHCA
    
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] <= 20,'p_ij_coverage'] = 1.0
    cp_ohca.loc[cp_ohca['dist_nearest_AED'] >= 100,'p_ij_coverage'] = 0.0
    cp_ohca.loc[(cp_ohca['dist_nearest_AED'] < 100) & (cp_ohca['dist_nearest_AED'] > 20),
                'p_ij_coverage'] = np.exp(- ALPHA * (cp_ohca['dist_nearest_AED'] - 20))
    # p_ij of each OHCA calulated
    # print(cp_ohca)
    
    return cp_ohca['p_ij_coverage'].mean()


In [ ]:
#totalCoverage(trainingOHCA, final_CC_df, MAX_DIST_METERS = 100)